In [2]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt 
import matplotlib as mpl
from matplotlib.dates import (YEARLY, DateFormatter,
                              rrulewrapper, RRuleLocator, drange)
import numpy as np
import os
import datetime
import pandas as pd
import xarray as xr
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from PIL import Image

root_dir='/glade/u/home/hongli/work/sharp/basins/scripts/'
os.chdir(root_dir)
output_folder='step4_plot_force_GMET_NLDAS_obs'
output_file = 'forcings_compare.png'
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

time_format='%Y-%m-%d %H:%M:%S'
formatter = DateFormatter('%m/%d/%y')
# formatter = DateFormatter('%b-%y') 
dpi_value = 100

# Read summa and NLDAS data
print('read')
nldas_force_file='/glade/u/home/hongli/work/sharp/basins/tayprk_huc12_nldas/forcings/forcing_nldas.tayprk.2010.nc'
gmet_force_file='/glade/u/home/hongli/work/sharp/basins/tayprk_huc12_metsim/forcings/gmet_3hr_summa/ens_forc.UPCO.huc12.001_20100101-20101231.nc'
p_vname = 'pptrate' #mm/s 
temp_vname = 'airtemp' #'K' 
t_vname = 'time'

# read nldas
f = xr.open_dataset(nldas_force_file)
p_nldas = f[p_vname].values[:] *24*3600 #mm/s to mm/day
temp_nldas = f[temp_vname].values[:]- 273.15 # K to C
time_str_nldas = f[t_vname].dt.strftime(time_format).values[:]
time_nldas = np.asarray([datetime.datetime.strptime(t, time_format) for t in time_str_nldas])
sim_start_str = time_str_nldas[0]
sim_end_str = time_str_nldas[-1]
f.close()

# read gmet
f = xr.open_dataset(gmet_force_file)
p_gmet = f[p_vname].values[:] *24*3600
temp_gmet = f[temp_vname].values[:]- 273.15 # K to C
time_str_gmet = f[t_vname].dt.strftime(time_format).values[:]
time_gmet = np.asarray([datetime.datetime.strptime(t, time_format) for t in time_str_gmet])
f.close()

# read daily observation
obs_file = os.path.join(root_dir, 'force_station.txt')
time_obs_str = np.loadtxt(obs_file, skiprows=1, dtype='str', usecols=[0])
# time_obs = [datetime.strptime(x, '%m/%d/%y') for x in data[:,0]]
data = np.loadtxt(obs_file, skiprows=1, usecols=[1,2,3])
data_value = np.zeros(np.shape(data))  #Max, Min, P
for i in range(np.shape(data)[0]):
    for j in range(np.shape(data)[1]):
        if data[i,j]==-999.0:
            data_value[i,j]=np.nan
        else:
            data_value[i,j]=data[i,j]        
t_obs_mean = ((data_value[:,0] + data_value[:,1])*0.5 - 32) * 5/9  # F to C
p_obs = np.asarray(data_value[:,2])*25.4 #inch/day to mm/day
    
# extract useful gmet and obs data
obs_time_start_index = np.where(time_obs_str == (time_nldas[0]).strftime('%Y-%m-%d'))[0][0]
obs_time_end_index = np.where(time_obs_str == (time_nldas[-1]).strftime('%Y-%m-%d'))[0][0]
p_obs = p_obs[obs_time_start_index:obs_time_end_index+1]
t_obs_mean = t_obs_mean[obs_time_start_index:obs_time_end_index+1]
time_obs_str = time_obs_str[obs_time_start_index:obs_time_end_index+1]

t_len = len(time_nldas)
p_gmet_part = np.zeros((t_len,3))
temp_gmet_part = np.zeros((t_len,3))
p_obs_part = np.zeros((t_len,1))
temp_obs_part = np.zeros((t_len,1))
for i in range(t_len):
    t = time_nldas[i]
    loc = np.where(time_gmet == t)[0]
    if not np.isnan(loc):
        p_gmet_part[i, :]=p_gmet[loc, :]
        temp_gmet_part[i, :]=temp_gmet[loc, :]
    else:
        p_gmet_part[i, :]=np.nan
        temp_gmet_part[i, :]=np.nan

    t_day = t.strftime('%Y-%m-%d')
    loc = np.where(time_obs_str == t_day)[0]
    if not np.isnan(loc):
        p_obs_part[i, 0]=p_obs[loc]
        temp_obs_part[i, 0]=t_obs_mean[loc]                          
    else:
        p_obs_part[i, 0]=np.nan
        temp_obs_part[i, 0]=np.nan                          
    print(t, time_obs_str[loc])
        
# plot
print('plot')
mksize = 0.25
alpah_value = 0.7
for hru in range(3):
    temp_fig_files = [] 
    print('hru '+str(hru+1))

    t_plot=[mpl.dates.date2num(hr) for hr in time_nldas]  

    row_num = 2
    fig, ax = plt.subplots(row_num,1)
    fig.set_figwidth(6.5*1.5) #190mm
    fig.set_figheight(6.5*0.4*row_num)

    for k in range(row_num):
        subplot_left = 0.25
        subplot_base_bottom = (row_num-k)*1.0/row_num
        subplot_width = 0.7
        subplot_hight = 0.75/row_num        

        if k == 0: # P
            ax[k].set_position([subplot_left, subplot_base_bottom-subplot_hight-0.04, subplot_width, subplot_hight]) #[left, bottom, width, height]         
            ax[k].plot_date(t_plot, p_gmet_part[:,hru], 'b-o', linewidth=0.75, markersize=mksize, alpha=alpah_value, label='GMET')  
            ax[k].plot_date(t_plot, p_nldas[:,hru], 'r-^', linewidth=0.75, markersize=mksize, alpha=alpah_value, label='NLDAS')                
            ax[k].plot_date(t_plot, p_obs_part, 'k-s', linewidth=0.75, markersize=mksize, alpha=alpah_value, label='Observation')                
            ylabel = 'Precipitation \n (mm/day)'

        else: # T
            ax[k].set_position([subplot_left, subplot_base_bottom-subplot_hight, subplot_width, subplot_hight]) #[left, bottom, width, height]         
            ax[k].plot_date(t_plot, temp_gmet_part[:,hru], 'b-o', linewidth=0.75, markersize=mksize, alpha=alpah_value, label='GMET')  
            ax[k].plot_date(t_plot, temp_nldas[:,hru], 'r-^', linewidth=0.75, markersize=mksize, alpha=alpah_value, label='NLDAS')
            ax[k].plot_date(t_plot, temp_obs_part, 'k-s', linewidth=0.75, markersize=mksize , alpha=alpah_value, label='Observation')
            ylabel = 'Temperature \n ($^\circ$C)'

        ax[k].xaxis.set_major_formatter(formatter)
        ax[k].xaxis.set_tick_params(labelsize='medium')#rotation=30, 
        fig.text(0.5*subplot_left, subplot_base_bottom-0.015-0.5*subplot_hight, ylabel, ha='center', va='center', rotation='horizontal')

        ax[k].legend(loc='upper left', fontsize='x-small', framealpha=0.5)
    
    output_filename = output_file.split('.')[0]+'_HRU'+str(hru+1)+'.png'
    fig.savefig(os.path.join(root_dir, output_folder, output_filename), dpi=dpi_value)
    plt.close(fig)                    

del t_plot, p_gmet, temp_gmet, p_gmet_part, temp_gmet_part,  p_nldas, temp_nldas 
print('Done')

read
2010-01-01 00:00:00 ['2010-01-01']
2010-01-01 03:00:00 ['2010-01-01']
2010-01-01 06:00:00 ['2010-01-01']
2010-01-01 09:00:00 ['2010-01-01']
2010-01-01 12:00:00 ['2010-01-01']
2010-01-01 15:00:00 ['2010-01-01']
2010-01-01 18:00:00 ['2010-01-01']
2010-01-01 21:00:00 ['2010-01-01']
2010-01-02 00:00:00 ['2010-01-02']
2010-01-02 03:00:00 ['2010-01-02']
2010-01-02 06:00:00 ['2010-01-02']
2010-01-02 09:00:00 ['2010-01-02']
2010-01-02 12:00:00 ['2010-01-02']
2010-01-02 15:00:00 ['2010-01-02']
2010-01-02 18:00:00 ['2010-01-02']
2010-01-02 21:00:00 ['2010-01-02']
2010-01-03 00:00:00 ['2010-01-03']
2010-01-03 03:00:00 ['2010-01-03']
2010-01-03 06:00:00 ['2010-01-03']
2010-01-03 09:00:00 ['2010-01-03']
2010-01-03 12:00:00 ['2010-01-03']
2010-01-03 15:00:00 ['2010-01-03']
2010-01-03 18:00:00 ['2010-01-03']
2010-01-03 21:00:00 ['2010-01-03']
2010-01-04 00:00:00 ['2010-01-04']
2010-01-04 03:00:00 ['2010-01-04']
2010-01-04 06:00:00 ['2010-01-04']
2010-01-04 09:00:00 ['2010-01-04']
2010-01-04 12:0

In [5]:
len(time_obs_str), len(time_obs)

(3653, 365)

In [4]:
time_obs_str

array(['2007-10-01', '2007-10-02', '2007-10-03', ..., '2017-09-28',
       '2017-09-29', '2017-09-30'], dtype='<U10')